# Assignment 1
The goals of this assignment are:
1. Familiarize yourselves with PYNQ board environment including
    Jupyter notebooks
    Inline C++ programming of the microblaze
    Python3 programming language
    PMOD peripherals as GPIO pins
2. Understand and build a functional Pulse Width Modulation (PWM) scheme to light an LED peripheral.
3. Discover the optimal PWM parameters relating to human perception.

Pulse Width Modulation
Without DAC, the GPIO at the output mode can only set the voltage at the specific pin with 2 states (LOW and HIGH). However, it is possible to ‘emulate’ an analog voltage with digital pins using a square wave. The electronics connected to the GPIO pin average the digital signal in time resulting in an analog voltage between (0-3.3V). One can modulate the width of square wave resulting in different averaged voltages (see figure below). This technique is called Pulse Width Modulation (PWM). With sufficient high PWM frequency (so that human will not be able to visually perceive the flashing), the brightness of an LED can be finely controlled by the duty cycle (the width of the square wave in percentage).

In [1]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

# Question 1 and 2

1. Use the gpio.ipynb from lab as a starting point. Add a C++ function to reset all the GPIO pins on the chosen PMOD.

2. Write a Python3 cell that emulates a PWM (for a chosen frequency and duty cycle) on one of the PMOD GPIO pins. For example, PMODB PIN3 needs to turn on for duty_cycle percent of the square wave frequency and off for the rest of the square wave. It may not be possible to achieve exactly 0% or 100% so be sure to add necessary code for those corner cases.

In [56]:
%%microblaze base.PMODA
#include "gpio.h"
#include "pyprintf.h"
#include <unistd.h>
static int inited = 0;
static gpio pins[8];
void init_pmoda()
{
    if(inited)
        return;
    for(int i = 0; i < 8; i++)
    { 
        pins[i] = gpio_open(i); 
        gpio_set_direction(pins[i], GPIO_OUT); 
        gpio_write(pins[i], 0); 
    } 
    inited = 1;   

}
//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val){ 
    if (val > 1){ 
        pyprintf("pin value must be 0 or 1"); 
    } 
    if(!inited) 
    { 
        init_pmoda(); 
    } 
    gpio_write(pins[pin], val);
}

//Function to read the value of a selected pin of PMODA
unsigned int read_gpio(unsigned int pin){ 
    gpio pin_in = gpio_open(pin); 
    gpio_set_direction(pin_in, GPIO_IN); 
    return gpio_read(pin_in);
}

// reset GPIO bins meaning writing 0 as output to all pins 0 - 7
void reset_pin(unsigned int pin)
{ 
    if(!inited) 
    { 
        init_pmoda(); 
    } 
    write_gpio(pin, 0); 
    //read_gpio(pin);
}

//-------------------------------------------------------Obsolete Code ---------------

//Function to turn on/off a selected pin of PMODA
/*void write_gpio(unsigned int pin, unsigned int val){ 
    if (val > 1){ 
        pyprintf("pin value must be 0 or 1"); 
    } 
    gpio pin_out = gpio_open(pin); 
    gpio_set_direction(pin_out, GPIO_OUT); 
    gpio_write(pin_out, val);
}
*/
//-----------------------------------------------------temp disable-------------------
/*
void run_pwm(unsigned int pin, 
             unsigned int freq_hz, 
             double duty_on, 
             double duration) 
{
    if (!inited) init_pmoda(); 
    if (pin >= 8) { pyprintf("pin must be 0-7\n"); return; }
    if (freq_hz == 0) { pyprintf("freq_hz must be > 0\n"); return; } 
    
    // Corner cases: 
    if (duty_on <= 0) {
        gpio_write(pins[pin], 0); 
        usleep(duration * 1000000.0); 
        return; 
    } 
    if (duty_on >= 1) { 
        gpio_write(pins[pin], 1); 
        usleep(duration * 1000000.0); 
        return; } 
    
    // Compute timing in microseconds 
    double period_s = 1.0 / (double)freq_hz; 
    
    double on_period_s = period_s * duty_on; 
    double off_period_s = period_s - on_period_s; 
    
    // did not find a time function as python's time.perf_counter(), so using cycle co 
    unsigned int cycles = duration / period_s; 
    
    for (unsigned int i = 0; i < cycles + 1; i++) { 
        gpio_write(pins[pin], 1); 
        usleep(on_period_s*1000000.0); 
        gpio_write(pins[pin], 0); 
        usleep(off_period_s*1000000.0); 
    }
}
*/
//------------------------------------------------------------------------------------
void run_pwm(unsigned int pin, 
             unsigned int freq_hz,
             unsigned int duty_milli, 
             unsigned int duration_ms)
{ 
    if (!inited) init_pmoda(); 
    
    if (pin >= 8) { pyprintf("pin must be 0-7\n"); return; } 
    if (freq_hz == 0) { pyprintf("freq_hz must be > 0\n"); return; } 
    if (duty_milli > 1000) duty_milli = 1000; 
    
    // corner cases: 0% / 100% 
    if (duty_milli == 0) { 
        gpio_write(pins[pin], 0); 
        usleep((useconds_t)duration_ms * 1000); 
        return; 
    } 
    
    if (duty_milli >= 1000) {
        gpio_write(pins[pin], 1); 
        usleep((useconds_t)duration_ms * 1000); 
        return;
        
    }

    // period in microseconds
    unsigned int period_us = 1000000u / freq_hz; // how many us per cycle
    if (period_us == 0) period_us = 1; // avoid 0 due to rounding ---- safety
    
    unsigned int on_us = (period_us * duty_milli) / 1000u;
    unsigned int off_us = period_us - on_us;

    pyprintf("on_us is %ui", on_us);
    pyprintf("on_us is %ui", off_us);

    // avoid 0 due to rounding ---- safety
    if (on_us == 0) on_us = 1;
    if (off_us == 0) off_us = 1;

    unsigned int cycles = (duration_ms * 1000u) / period_us;

    for (unsigned int i = 0; i < cycles + 1; i++) {
        gpio_write(pins[pin], 1);
        usleep((useconds_t)on_us);
        gpio_write(pins[pin], 0);
        usleep((useconds_t)off_us);
    }
}

void run_pwm_for6(unsigned int pin)
{
    run_pwm(pin, 100, 250, 1000);
    write_gpio(pin, 0);
    usleep((useconds_t)1000000u);
}

# Question 3

Find the optimal PWM frequency, such that the physical flashing phenomenon will not be perceived visually;

In [15]:
for pin_num in range(8): 
    reset_pin(pin_num);
#write_gpio(1, 1)

# run_pwm(pin, frequency, duty cycle, duration)

# 10Hz PWM Frequency (Blinking)
run_pwm(2, 10, 988, 5000) 

In [16]:
# 20Hz PWM Frequency (Blinking)
run_pwm(2, 20, 988, 5000) 

In [17]:
# 30Hz PWM Frequency (Blinking)
run_pwm(2, 30, 988, 5000) 

In [18]:
# 40Hz PWM Frequency (Flashing Hard to Detect)
run_pwm(2, 40, 988, 5000) 

In [19]:
# 50Hz PWM Frequency (Flashing Almost Negligible)
run_pwm(2, 50, 988, 5000) 

In [20]:
# 60Hz PWM Frequency (Flashing Almost Negligible)
run_pwm(2, 60, 988, 5000) 

In [21]:
# 70Hz PWM Frequency (Flashing Almost Negligible)
run_pwm(2, 70, 988, 5000) 

In [22]:
# 80Hz PWM Frequency (No Flashing)
run_pwm(2, 80, 988, 5000) 

In [23]:
# 75Hz PWM Frequency (Ideal Frequency)
run_pwm(2, 75, 988, 5000)

# summary for report : 
# 10Hz is blinking, so as 30 
# 40 is hard to detect, 
# 50 is almost neglectable
# 75 is like normal

# Question 4

Achieve the visually perceived 100%, 75%, 50% and 25% of full LED brightness by adjusting the duty cycle;

In [41]:
# 100% LED brightness
run_pwm(2, 100, 1000, 2000) 

In [42]:
# 75% # LED brightness
run_pwm(2, 100, 750, 2000) 

In [43]:
# 50% LED brightness
run_pwm(2, 100, 500, 2000) 

In [44]:
# 25% LED brightness
run_pwm(2, 100, 250, 2000) 

# Question 5

Varying the duty cycles and approximate the corresponding LED brightness (in the unit of %). To the end, plot and explain the approximate relationship of % brightness versus duty cycle. While working on this task, feel free to check out the Weber-Fechner law. For light perception, this law describe the observation that the our eyes perceive light in a non-linear way.

In [225]:
# 100% Duty Cycle
run_pwm(2, 100, 1000, 2000) 

In [226]:
# 90% Duty Cycle
run_pwm(2, 100, 900, 2000) 

In [227]:
# 80% Duty Cycle
run_pwm(2, 100, 800, 2000) 

In [228]:
# 70% Duty Cycle
run_pwm(2, 100, 700, 2000) 

In [229]:
# 60% Duty Cycle
run_pwm(2, 100, 600, 2000) 

In [230]:
# 50% Duty Cycle
run_pwm(2, 100, 500, 2000) 

In [231]:
# 40% Duty Cycle
run_pwm(2, 100, 400, 2000) 

In [232]:
# 30% Duty Cycle
run_pwm(2, 100, 300, 2000)

In [195]:
# 20% Duty Cycle
run_pwm(2, 100, 200, 2000) 

In [210]:
# 10% Duty Cycle
run_pwm(2, 100, 100, 2000) 

# Question 6

After you have experimented with various PWM frequencies and duty cycles, add the following functionalities for a fixed duty cycle (i.e. 25%). You’ll want to use asyncio for this part. 

Start the code and blink the LED’s red channel in intervals of 1 second (i.e. 1 second on, 1 second off)
When buttons 0, 1, or 2 are pushed, the LED will change color from Red, to Green, to Blue.
When button 3 is pushed, the LED will stop blinking.
Your video should demonstrate how each button change to each of the (4) colors. 

In [57]:
btns = base.btns_gpio

In [59]:
import asyncio
RED = 1
GREEN = 2
BLUE = 3
STOP = 0
led_blink_color = RED
async def flash_leds():
    global led_blink_color, RED, GREEN, BLUE, STOP
    while 1:
        await asyncio.sleep(2)
        if led_blink_color == RED:
            #await asyncio.to_thread(run_pwm, RED, 0.25, 1, 100)
            #run_pwm(RED, 100, 250, 1000)
            #write_gpio(RED, 0)
            #await asyncio.sleep(1)
            run_pwm_for6(RED)
        elif led_blink_color == GREEN:
            #await asyncio.to_thread(run_pwm, GREEN, 0.25, 1, 100)
            #run_pwm(GREEN, 100, 250, 1000)
            #write_gpio(GREEN, 0)
            #await asyncio.sleep(1)
            run_pwm_for6(GREEN)
        elif led_blink_color == BLUE:
            #await asyncio.to_thread(run_pwm, BLUE, 0.25, 1, 100)
            #run_pwm(BLUE, 100, 250, 1000)
            #write_gpio(BLUE, 0)
            #await asyncio.sleep(1)
            run_pwm_for6(BLUE)
        elif led_blink_color == STOP:
            for pin_num in range(8):
                reset_pin(pin_num)
            break

async def get_btns(_loop):
    global led_blink_color, RED, GREEN, BLUE, STOP
    while 1:
        await asyncio.sleep(0.01) # releasing resources for other tasks to run
        v = btns.read()
        if v & 0x1: # button 0 RED
            led_blink_color = RED
            print(led_blink_color)
            # await asyncio.sleep(0.05) # releasing resources for other tasks to run
        elif v & 0x2: # button 1 GREEN
            led_blink_color = GREEN
            print(led_blink_color)
            # await asyncio.sleep(0.05) # releasing resources for other tasks to run
        elif v & 0x4: # button 2 BLUE
            led_blink_color = BLUE
            print(led_blink_color)
            # await asyncio.sleep(0.05) # releasing resources for other tasks to run
        elif v & 0x8: # button 3 STOP
            led_blink_color = STOP
            print(led_blink_color)
            _loop.stop()
            break

# async def main():
# await asyncio.gather(flash_leds(), get_btns())

# await main()

loop = asyncio.new_event_loop()
loop.create_task(flash_leds())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()

print("Done.")        

1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
Done.
